# Wokflow  **TEST** zLightGBM

##  Big Picture

Este workflow es de testeo, hace la predicción final en 202106 el último mes para el cual podemos construir la clase_ternaria completa y por lo tanto medir la ganancia.

Se muestra el workflow con la Bayesian Optimization diseñada para que LightGBM maximice la ganancia en pesos argentinos
<br> En la Primera Competencia se maximizo la metrica global ROC AUC,  ahora pasamos a la metrica real que llamamos ganancia y que solo se concentra en los  11000 registros con mayor probabilidad de  {"BAJA+1","BAJA+2"}


Intencionalmente este notebook es un *semi-esqueleto*  al que usted deberá completar reutilizando código que ya generó para la Primera Competencia y nuevo código que resuelva las problemáticas presentes en esta competencia.

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-12 18:45:19 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658633,35.2,1454648,77.7,1227766,65.6
Vcells,1222301,9.4,8388608,64.0,1975147,15.1


In [2]:
source("/home/vickydiliscia/dmeyf2025/src/churn/C2002/config-APO2002-C202106.r")

In [3]:
source(PARAM$utils$promedios)
source(PARAM$utils$ratios)

In [4]:
source(file.path(PARAM$paths$functions, "index.R"))
source_functions(PARAM$paths$functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: crear_proporcion.R

Sourcing: interpolar_faltantes.R

Sourcing: suma_segura.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [5]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [6]:
Sys.time()
require( "data.table" )
# leo el dataset
dataset <- fread(PARAM$paths$datasetcrudo)

[1] "2025-11-12 18:45:29 UTC"

In [7]:
dataset <- agregar_clase_ternaria(dataset)

In [8]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [9]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

In [10]:
dq <- ver_meses_rotos(dataset, month_col = "foto_mes", exclude = c("numero_de_cliente", "clase_ternaria"))

salida_ceros <- dq$detalle[
  pct_zero == 1, 
  .(
    meses_100_cero = .N,
    foto_mes_cero  = paste(sort(unique(foto_mes)), collapse = ",")
  ),
  by = variable
][order(-meses_100_cero, variable)]

salida_ceros[]# Por si 'variable' vino como factor
salida_ceros[, variable := as.character(variable)]

variable,meses_100_cero,foto_mes_cero
<fct>,<int>,<chr>
ccajeros_propios_descuentos,6,"201910,202002,202006,202009,202010,202102"
mcajeros_propios_descuentos,6,"201910,202002,202006,202009,202010,202102"
ctarjeta_visa_descuentos,6,"201910,202002,202006,202009,202010,202102"
mtarjeta_visa_descuentos,6,"201910,202002,202006,202009,202010,202102"
ctarjeta_master_descuentos,6,"201910,202002,202006,202009,202010,202102"
mtarjeta_master_descuentos,6,"201910,202002,202006,202009,202010,202102"
mrentabilidad,3,"201905,201910,202006"
mrentabilidad_annual,3,"201905,201910,202006"
mcomisiones,3,"201905,201910,202006"


In [11]:
variables_a_interpolar <- setNames(
  lapply(
    salida_ceros[["foto_mes_cero"]],
    function(s) as.integer(trimws(unlist(strsplit(s, ","))))
  ),
  salida_ceros[["variable"]]
)

# Opcional: ver estructura rápida
# str(variables_a_interpolar)

# Opcional: imprimir en formato “lindo” con sufijo L
pretty_print_list <- function(lst) {
  lines <- mapply(function(nm, v) {
    paste0("  ", nm, " = c(", paste0(paste0(v, "L"), collapse = ", "), "),")
  }, names(lst), lst, SIMPLIFY = TRUE, USE.NAMES = FALSE)
  cat("list(\n", paste(lines, collapse = "\n"), "\n)\n", sep = "")
}

pretty_print_list(variables_a_interpolar)

list(
  ccajeros_propios_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  mcajeros_propios_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  ctarjeta_visa_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  mtarjeta_visa_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  ctarjeta_master_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  mtarjeta_master_descuentos = c(201910L, 202002L, 202006L, 202009L, 202010L, 202102L),
  mrentabilidad = c(201905L, 201910L, 202006L),
  mrentabilidad_annual = c(201905L, 201910L, 202006L),
  mcomisiones = c(201905L, 201910L, 202006L),
  mactivos_margen = c(201905L, 201910L, 202006L),
  mpasivos_margen = c(201905L, 201910L, 202006L),
  ccomisiones_otras = c(201905L, 201910L, 202006L),
  mcomisiones_otras = c(201905L, 201910L, 202006L),
  chomebanking_transacciones = c(201910L, 202006L),
  ccajas_depositos = c(202006L, 202105L),
  active_quarter = c(

In [12]:
dataset <- interpolar_faltantes(
  dataset,
  variables = variables_a_interpolar,
  treat_zero_as_na = TRUE,
  require_both     = TRUE,
  only_if_na       = TRUE,
  integer_prefix   = "^c",   # deja enteras todas las que empiezan con 'c'
  round_integers   = TRUE    # y además respeta integer si la original lo era
)

### Data Drifting

Se debe corregir el drifting natural que ocurre en loa datos, en particular los datos monetarios que se vieron fuertemente afectados por una alta inflación
<br> Posibles métodos son:
* No hacer absolutamente nada
* Ajuste de valores monetarios por indices del tipo :
   * IPC  Indice de Precios al Consumidor
   * Dolar Oficial
   * Dolar Blue
   * UVA  Unidad de Valor Adquisitivo

a este codigo de Data Drifting lo debera escribir usted

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [13]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-11-12 18:55:50 UTC"

In [14]:
dataset <- copy(dataset)[, `:=`(
  mtarjeta_total_consumo              = suma_segura(mtarjeta_visa_consumo, mtarjeta_master_consumo),
  mtarjeta_total_debitos_automaticos  = suma_segura(mttarjeta_visa_debitos_automaticos, mttarjeta_master_debitos_automaticos),
  mfinanciacionlimite_total           = suma_segura(Master_mfinanciacion_limite, Visa_mfinanciacion_limite),
  msaldopesos_total                   = suma_segura(Master_msaldopesos, Visa_msaldopesos),
  msaldodolares_total                 = suma_segura(Master_msaldodolares, Visa_msaldodolares),
  mconsumopesos_total                 = suma_segura(Master_mconsumospesos, Visa_mconsumospesos),
  mconsumodolares_total               = suma_segura(Master_mconsumosdolares, Visa_mconsumosdolares),
  mlimitecompra_total                 = suma_segura(Master_mlimitecompra, Visa_mlimitecompra),
  madelantopesos_total                = suma_segura(Master_madelantopesos, Visa_madelantopesos),
  madelantodolares_total             = suma_segura(Master_madelantodolares, Visa_madelantodolares),
  mpagado_total                       = suma_segura(Master_mpagado, Visa_mpagado),
  mpagopesos_total                    = suma_segura(Master_mpagospesos, Visa_mpagospesos),
  mpagosdolares_total                 = suma_segura(Master_mpagosdolares, Visa_mpagosdolares),
  msaldototal_total                   = suma_segura(Master_msaldototal, Visa_msaldototal),
  mconsumototal_total                 = suma_segura(Master_mconsumototal, Visa_mconsumototal),
  cconsumos_total                     = suma_segura(Master_cconsumos, Visa_cconsumos),
  delinquency_total                   = suma_segura(Master_delinquency, Visa_delinquency),

  mplazofijo_total                    = suma_segura(mplazo_fijo_dolares, mplazo_fijo_pesos),
  mpayroll_total                      = suma_segura(mpayroll, mpayroll2),
  minversion1_total                   = suma_segura(minversion1_dolares, minversion1_pesos),
  cpayroll_total                      = suma_segura(cpayroll_trx, cpayroll2_trx),
  cseguros_total                      = suma_segura(
                                          suma_segura(
                                            suma_segura(cseguro_vida, cseguro_auto),
                                            cseguro_vivienda
                                          ),
                                          cseguro_accidentes_personales
                                        )
)]

dataset <- copy(dataset)[, `:=`(
  madelanto_total = suma_segura(madelantodolares_total, madelantopesos_total)
)]

### Feature Engineering Historico

In [15]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [16]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [17]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [18]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-12 18:57:29 UTC"

In [19]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [20]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1059

[1] "2025-11-12 18:58:32 UTC"

### Feature Engineering a partir de hojas de Random Forest

In [21]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm



In [22]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}

In [23]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [24]:
# Feature Engineering agregando variables de Random Forest
#  aqui es donde se hace el trabajo
AgregaVarRandomForest()

Sys.time()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  201901 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201902 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201903 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201904 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201905 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201906 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201907 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201908 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

[1] "2025-11-12 19:01:55 UTC"

In [25]:
ncol(dataset)
colnames(dataset)

[1] 1380

[1] "numero_de_cliente"                          
   [2] "foto_mes"                                   
   [3] "active_quarter"                             
   [4] "cliente_vip"                                
   [5] "internet"                                   
   [6] "cliente_edad"                               
   [7] "cliente_antiguedad"                         
   [8] "mrentabilidad"                              
   [9] "mrentabilidad_annual"                       
  [10] "mcomisiones"                                
  [11] "mactivos_margen"                            
  [12] "mpasivos_margen"                            
  [13] "cproductos"                                 
  [14] "tcuentas"                                   
  [15] "ccuenta_corriente"                          
  [16] "mcuenta_corriente_adicional"                
  [17] "mcuenta_corriente"                          
  [18] "ccaja_ahorro"                               
  [19] "mcaja_ahorro"                               
  [20] "mcaja_ahorro_adicional"                     
  [21] "mcaja_ahorro_dolares"                       
  [22] "cdescubierto_preacordado"                   
  [23] "mcuentas_saldo"                             
  [24] "ctarjeta_debito"                            
  [25] "ctarjeta_debito_transacciones"              
  [26] "mautoservicio"                              
  [27] "ctarjeta_visa"                              
  [28] "ctarjeta_visa_transacciones"                
  [29] "mtarjeta_visa_consumo"                      
  [30] "ctarjeta_master"                            
  [31] "ctarjeta_master_transacciones"              
  [32] "mtarjeta_master_consumo"                    
  [33] "cprestamos_prendarios"                      
  [34] "mprestamos_prendarios"                      
  [35] "cprestamos_hipotecarios"                    
  [36] "mprestamos_hipotecarios"                    
  [37] "cplazo_fijo"                                
  [38] "mplazo_fijo_dolares"                        
  [39] "mplazo_fijo_pesos"                          
  [40] "cinversion1"                                
  [41] "minversion1_pesos"                          
  [42] "minversion1_dolares"                        
  [43] "cinversion2"                                
  [44] "minversion2"                                
  [45] "cseguro_vida"                               
  [46] "cseguro_auto"                               
  [47] "cseguro_vivienda"                           
  [48] "cseguro_accidentes_personales"              
  [49] "ccaja_seguridad"                            
  [50] "cpayroll_trx"                               
  [51] "mpayroll"                                   
  [52] "mpayroll2"                                  
  [53] "cpayroll2_trx"                              
  [54] "ccuenta_debitos_automaticos"                
  [55] "mcuenta_debitos_automaticos"                
  [56] "ctarjeta_visa_debitos_automaticos"          
  [57] "mttarjeta_visa_debitos_automaticos"         
  [58] "ctarjeta_master_debitos_automaticos"        
  [59] "mttarjeta_master_debitos_automaticos"       
  [60] "cpagodeservicios"                           
  [61] "mpagodeservicios"                           
  [62] "cpagomiscuentas"                            
  [63] "mpagomiscuentas"                            
  [64] "ccajeros_propios_descuentos"                
  [65] "mcajeros_propios_descuentos"                
  [66] "ctarjeta_visa_descuentos"                   
  [67] "mtarjeta_visa_descuentos"                   
  [68] "ctarjeta_master_descuentos"                 
  [69] "mtarjeta_master_descuentos"                 
  [70] "ccomisiones_mantenimiento"                  
  [71] "mcomisiones_mantenimiento"                  
  [72] "ccomisiones_otras"                          
  [73] "mcomisiones_otras"                          
  [74] "cforex"                                     
  [75] "cforex_buy"                                 
  [76] "mforex_buy"         

## Optimizacion de Hipeparámetros

No hacemos optimizacion

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en todos los meses salvo los de pandemia esta es una meticulosa decisión.
* Se hace un muy consevador undersampling del **0.50** = 50% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **100** canaritos, esta es una decisión enorme !

In [26]:
PARAM$train_final$future <- c(202106)

PARAM$train_final$meses <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104
)

PARAM$train_final$undersampling <- 0.50

In [27]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [28]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [29]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model zLGBM

In [30]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm

Registered S3 methods overwritten by 'zlightgbm':
  method                 from    
  dimnames<-.lgb.Dataset lightgbm
  dim.lgb.Dataset        lightgbm
  dimnames.lgb.Dataset   lightgbm
  predict.lgb.Booster    lightgbm
  print.lgb.Booster      lightgbm
  summary.lgb.Booster    lightgbm


Attaching package: ‘zlightgbm’


The following objects are masked from ‘package:lightgbm’:

    get_field, getLGBMthreads, lgb.configure_fast_predict,
    lgb.convert_with_rules, lgb.cv, lgb.Dataset, lgb.Dataset.construct,
    lgb.Dataset.create.valid, lgb.Dataset.save,
    lgb.Dataset.set.categorical, lgb.Dataset.set.reference,
    lgb.drop_serialized, lgb.dump, lgb.get.eval.result, lgb.importance,
    lgb.interprete, lgb.load, lgb.make_serializable, lgb.model.dt.tree,
    lgb.plot.importance, lgb.plot.interpretation, lgb.restore_handle,
    lgb.save, lgb.slice.Dataset, lgb.train, lightgbm, set_field,
    setLGBMthreads




[1] "2025-11-12 19:03:38 UTC"

In [31]:
cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-11-12 19:04:07 UTC"

In [32]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

Nuevos hiperparámetros en zLightGBM

canaritos , entero >=0, cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
gradient_bound , numero real, >=0 cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo
En el caso que canaritos==0 y gradient_bound==0 entonces xLightGBM se comporta exactamente igual a LightGBM

Tener en cuenta que el hiperparámetro min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
realmente valdria la pena experimentar con distintos valores de min_data_in_leaf, hay potencial de mejora !

####  Entrenamiento del modelo

Aqui se ejecuta zLightGBM
no se utilizan hiperparámetros optimos para controlar el overfitting
zLightGBM sabe cuando no hacer un split
si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina

claramente el min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [ ]:
nrow(dtrain_final)  # cantidad de filas (observaciones)
ncol(dtrain_final)  # cantidad de columnas (variables)
dim(dtrain_final)   # ambas a la vez

In [ ]:
PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 20L,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.15  # default de zLightGBM
)


In [ ]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

In [ ]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
modelo_file <- paste0(PARAM$paths$models , "/zmodelo.txt")
lgb.save(modelo_final, file=modelo_file)

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
su mmary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

### Scoring

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [ ]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

## Clasificacion

In [ ]:
Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0(PARAM$paths$kaggle, PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [61]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:icerca, Predicted := 1L] # marco los primeros

archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_",  icerca, ".csv")

# grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_pseudo_kaggle,
  sep= ","
)


In [62]:
# la subida a Kaggle
comando <- "kaggle competitions submit"
competencia <- "-c  test-202106"
arch <- paste( "-f", archivo_pseudo_kaggle)
mensaje <-  paste0( "-m 'exp=", PARAM$experimento,
  " envios=", icorte_mejor,"'")

                    
linea <- paste( comando, competencia, arch, mensaje)
salida <- system(linea, intern=TRUE)
cat(salida)

Successfully submitted to Test202106

In [63]:
Sys.time()

[1] "2025-11-07 21:17:35 EST"

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [45]:
# Semillerio Final
PARAM$train_final$APO <- 100
PARAM$train_final$ksemillerio  <- 100

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)

In [46]:
# valvula de seguridad anti atajos
if( PARAM$train_final$APO * PARAM$train_final$ksemillerio  < 100
  | PARAM$train_final$APO < 5
)  {
  # stop("No quiera overfitear")
  cat("No quiera overfitear")
}

In [47]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )]
PARAM$train_final$semillas

[1] 974411 507673 696271 281233 677309 180647 251519 235241 176213 162601
 [11] 138637 519551 807931 471301 152063 823663 874337 466201 206191 787939
 [21] 361327 320141 459847 246833 208799 381371 868327 745187 297371 244147
 [31] 775739 486043 903163 228989 714223 766943 838157 492253 954757 698393
 [41] 424103 964637 581239 315739 868379 589439 114487 645131 794953 356567
 [51] 609821 660799 957059 495629 227627 647161 499327 671837 810239 776219
 [61] 205663 681781 766739 671633 450601 204583 822839 217411 333071 823033
 [71] 631471 595073 139663 564301 459749 713863 979171 119569 935303 440641
 [81] 207187 907811 552263 281839 990281 326707 346739 451783 523771 377129
 [91] 783613 720901 268843 349409 352619 473353 847601 859787 427241 365201

In [48]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 437055 columnas 1241 


[1] "2025-11-07 21:08:58 EST"

In [49]:
# genero los modelitos

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0(PARAM$paths$models, "/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:08:58 EST"

Se hace el predict() del modelo en los datos del futuro

In [50]:
# dataset de future, donde en este caso estoy haciendo testing
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [51]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [52]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0(PARAM$paths$models, "/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:17:29 EST"

In [53]:
mganancias

407600000,411200000,406800000,404000000
407600000,406400000,408400000,407200000
410800000,408800000,406800000,405600000
407600000,407200000,403600000,403200000
411600000,411200000,407600000,407200000
406000000,402400000,404400000,405600000
407600000,412000000,408400000,408000000
405200000,407200000,409200000,404000000
407600000,405600000,407600000,404800000
408400000,409600000,407600000,403200000


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [54]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [55]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 11000

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:17:31 EST"

## Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [56]:
colmedias <- colMeans( mganancias, na.rm=TRUE )
mcorte_mejor <- max(colmedias, na.rm=TRUE)
icorte_mejor <- which.max( colmedias )
corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [57]:
tbl <- as.data.table( as.list( colmedias ) )
colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
tbl[, experimento := PARAM$experimento ]

dir.create(exp_gral, showWarnings=FALSE)
fwrite( tbl,
  file= paste0( PARAM$paths$apo, "/tb_experimentos.txt"),
  sep= "\t",
  append= TRUE
)

In [58]:
colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
tbl_local <- as.data.table( mganancias )

fwrite( tbl_local,
  file= "tb_apo.txt",
  sep= "\t"
)

In [59]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

mcorte_mejor
icerca
tb_prediccion[ icerca]

[1] 408160000

[1] 10032

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
1209477280,202106,1,0.113957,408160000


In [60]:
icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
icerca

[1] 10032